## OOP Revision

## Building Custom Exceptions

A `custom exception` is just a **class that inherits** from Python’s built-in `Exception`.

In [1]:
class MyCustomException(Exception):
    """A simple custom exception"""
    
    pass 

try:
    raise MyCustomException("Somethig went wrong")
except MyCustomException as e:
    print(f"Caught custom error: {e}")

Caught custom error: Somethig went wrong


In [2]:
class MyCustomError(Exception):
    def __init__(self, message, code):
        super().__init__(message)
        self.code = code

try:
    raise MyCustomError("Invalid data format!", code=400)
except MyCustomError as e:
    print(f"Error: {e}, Code: {e.code}")


Error: Invalid data format!, Code: 400


- `class MyCustomError(Exception):`
   - You're inheriting from Python’s built-in Exception class.
   - This means MyCustomError behaves like a normal Python error but with extra features.

- `__init__(self, message, code):`
  - This is the constructor. It initializes the error object with message and code.

- `super().__init__(message):`
  - Calls the parent Exception class’s constructor.
  - This ensures message is stored correctly so Python can display it when the exception is raised.

- `self.code = code:`
  - Adds an extra attribute code to the exception object.
  - This is custom — normal exceptions don’t have this unless you define it.

In [3]:
class InvalidAgeError(Exception):
    def __init__(self, age, message="Age must be between 18 and 60"):
        self.age = age
        self.message = message
        super().__init__(f"{message}. Received: {age}")

try:
    age = 15
    if age < 18 or age > 60:
        raise InvalidAgeError(age)
except InvalidAgeError as e:
    print(f"Error: {e}")

Error: Age must be between 18 and 60. Received: 15


In [5]:
class InvalidAgeError(Exception):
    def __init__(self, age, message="Age must be between 18 and 60 but can't be float"):
        self.age = age
        self.message = message
        super().__init__(f"{message}. Received: {age}")

try:
    age = 18.5
    if age < 18 or age > 60 or not isinstance(age, int):
        raise InvalidAgeError(age)
except InvalidAgeError as e:
    print(f"Error: {e}")

Error: Age must be between 18 and 60 but can't be float. Received: 18.5


In [6]:
class InvalidAgeError(Exception):
    def __init__(self, age, message="Age must be between 18 and 60 but can't be float"):
        self.age = age
        self.message = message
        super().__init__(f"{message}. Received: {age}")

try:
    age = 45
    if age < 18 or age > 60 or not isinstance(age, int):
        raise InvalidAgeError(age)
    print(f"Age is properly added and is {age}")
except InvalidAgeError as e:
    print(f"Error: {e}")

Age is properly added and is 45


#### Inheriting from Different Base Classes

Depending on the context, you might want to inherit from:
- `Exception` → general custom errors 
- `ValueError` → invalid input
- `RuntimeError` → runtime failures
- `IOError` → I/O-related errors

Example

```python
class InvalidEmailError(ValueError):
    pass

```

In [ ]:
class NegativeValueError(ValueError):
    """Raised when a negative value is provided where it's not allowed."""
    pass

def deposit(amount):
    if amount < 0:
        raise NegativeValueError("Amount cannot be negative!")
    print(f"Deposited: {amount}")

try:
    deposit(-500)
except NegativeValueError as e:
    print(f"Caught: {e}")
except ValueError as e:
    print(f"Caught generic ValueError: {e}")

Caught: Amount cannot be negative!


In [10]:
age = 45

print(type(age).__name__)

int


In [13]:
class InvalidTypeError(TypeError):
    """Raised when an argument is not of the expected type."""
    def __init__(self, expected_type, received_type):
        super().__init__(f"Expected {expected_type}, got {received_type}")

def set_age(age):
    if not isinstance(age, int):
        raise InvalidTypeError(int, type(age).__name__)
    print(f"Age set to {age}")

try:
    set_age("twenty")
except InvalidTypeError as e:
    print(f"Caught: {e}")


Caught: Expected <class 'int'>, got str


In [15]:
class MissingConfigKeyError(KeyError):
    """Raised when a required configuration key is missing."""
    def __init__(self, key):
        super().__init__(f"The required config key '{key}' is missing.")

config = {"host": "localhost", "port": 5432}

def get_config(key):
    if key not in config:
        raise MissingConfigKeyError(key)
    return config[key]

try:
    print(get_config("password"))
except MissingConfigKeyError as e:
    print(f"Caught: {e}")

Caught: "The required config key 'password' is missing."


In [16]:
# Custom file Handling error
class FileNotAccessibleError(IOError):
    """Raised when a file cannot be accessed."""
    def __init__(self, filename):
        super().__init__(f"File '{filename}' cannot be accessed.")

import os

def read_file(filename):
    if not os.path.exists(filename):
        raise FileNotAccessibleError(filename)
    with open(filename, "r") as file:
        return file.read()

try:
    read_file("data.txt")
except FileNotAccessibleError as e:
    print(f"Caught: {e}")

Caught: File 'data.txt' cannot be accessed.


#### Integrating Logging for Better Exceptions

In [17]:
import logging

logging.basicConfig(level=logging.ERROR)

class PaymentError(Exception):
    def __init__(self, message):
        super().__init__(message)
        logging.error(f"Payment Error: {message}")

try:
    raise PaymentError("Payment declined due to insufficient funds.")
except PaymentError as e:
    print(e)

ERROR:root:Payment Error: Payment declined due to insufficient funds.


Payment declined due to insufficient funds.


In [18]:
class APIError(Exception):
    def __init__(self, status_code, message, endpoint):
        self.status_code = status_code
        self.endpoint = endpoint
        super().__init__(f"[{status_code}] {message} at {endpoint}")

try:
    raise APIError(404, "Resource not found", "/users/42")
except APIError as e:
    print(e)

[404] Resource not found at /users/42


In [19]:
## Raissing and chaining exceptions

try:
    int("abc")
except ValueError as e:
    raise APIError(400, "Invalid input type", "/parse") from e

APIError: [400] Invalid input type at /parse

#### Integration with Fast API

```python
from fastapi import FastAPI, HTTPException

app = FastAPI()

class UserNotFoundError(Exception):
    def __init__(self, user_id):
        self.user_id = user_id
        self.message = f"User {user_id} not found"
        super().__init__(self.message)

@app.exception_handler(UserNotFoundError)
async def user_not_found_handler(request, exc: UserNotFoundError):
    return JSONResponse(status_code=404, content={"error": exc.message})

@app.get("/users/{user_id}")
async def get_user(user_id: int):
    if user_id != 1:
        raise UserNotFoundError(user_id)
    return {"id": 1, "name": "John Doe"}
```

### Using `sys.exc_info()` for Debugging

#### sys advantage

- `sys.exc_info()` is a function that **returns information about the most recent exception** that is currently being handled.
- It returns a tuple: 
  ```python
  (exception_type, exception_value, traceback_object)
  ```

- `exception_type` → the class of the exception
- `exception_value` → the actual exception instance (e.g., ZeroDivisionError('division by zero'))
- `traceback_object` → the traceback object that contains details like file name, line number, and function.

In [20]:
import sys

try:
    1 / 0 # type: ignore
except ZeroDivisionError:
    print(sys.exc_info())

(<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero'), <traceback object at 0x000001A0EC75AE80>)


In [22]:
import sys

try:
    1 / 0 # type: ignore
except ZeroDivisionError:
    print(sys.exc_info()[0].__name__) # type: ignore

ZeroDivisionError


> If no exception is being handled, sys.exc_info() returns:

```python
(None, None, None)
```

In [ ]:
import sys

try:
    1 / 0 # type: ignore
except ZeroDivisionError:
    _, _, exc_tb = sys.exc_info()
    file_name = exc_tb.tb_frame.f_code.co_filename # type: ignore # From the traceback object, it grabs the filename where the exception occurred.
    error_message = (
    f"Error occurred in python script [{file_name}] "
    f"at line [{exc_tb.tb_lineno}] " # type: ignore
    )
    print(error_message)

Error occurred in python script [C:\Users\user\AppData\Local\Temp\ipykernel_33768\4189403364.py] at line [4] 


In [ ]:
import sys
import traceback

def error_message_detail(error, error_detail=sys):
    exc_type, exc_value, exc_tb = error_detail.exc_info()
    
    if exc_tb is None:
        # No active exception, return a generic message
        return f"No active exception. Message: {str(error)}"
    
    file_name = exc_tb.tb_frame.f_code.co_filename
    line_no = exc_tb.tb_lineno
    stack_trace = ''.join(traceback.format_exception(exc_type, exc_value, exc_tb))
    
    return (
        f"Error in script [{file_name}] at line [{line_no}]: {error}\n"
        f"Traceback:\n{stack_trace}"
    )

try:
    1 / 0 # type: ignore
except Exception as e:
    print(error_message_detail(e))

Error in script [C:\Users\user\AppData\Local\Temp\ipykernel_33768\846706352.py] at line [22]: division by zero
Traceback:
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_33768\846706352.py", line 22, in <module>
    1 / 0
ZeroDivisionError: division by zero



#### Hierarchy

```markdown
BaseException
 ├── Exception
 │    ├── ValueError
 │    ├── TypeError
 │    ├── KeyError
 │    └── ...
 └── SystemExit, KeyboardInterrupt, etc.
```

In [ ]:
import logging

class AppError(Exception): pass

def risky_function():
    try:
        1 / 0 # type: ignore
    except ZeroDivisionError as e:
        logging.error("Math error", exc_info=True)
        raise AppError("Calculation failed") from e

In [27]:
risky_function()

ERROR:root:Math error
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_33768\481723666.py", line 7, in risky_function
    1 / 0
ZeroDivisionError: division by zero


AppError: Calculation failed